In [105]:
#dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from pandas import json_normalize
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import requests
import json

In [123]:
def atlanta_df():
    atl_crime_df=pd.read_csv("static/data/atlanta_crime_2020.csv")

    new_atl_crime_df=atl_crime_df.drop(['occur_date', 'occur_time', 'poss_date','poss_time', 'beat', 'apartment_office_prefix', 'apartment_number','watch','location_type','UCR_Number','npu'],axis=1)

    new_atl_crime_df=new_atl_crime_df.rename(columns={'offense_id':'id','location':'block','lat':'latitude','long':'longitude','neighborhood':'location_description','UC2_Literal':'primary_type'})

    new_atl_crime_df=new_atl_crime_df

    indexNames=new_atl_crime_df[(new_atl_crime_df['latitude']==0)].index



    crime_atl_df=new_atl_crime_df.drop(indexNames).iloc[:964]
    return crime_atl_df.set_index('id')

In [127]:
def chicago_df():
    url = 'https://data.cityofchicago.org/resource/qzdf-xmn8.json'
    # reverseGeoAPI = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{lng}%2C%20{lt}.json?access_token={API_KEY}"

    df = pd.read_json(url)
    df.drop_duplicates("id", inplace=True)

    new_df =df.set_index("id")
    new_df.dropna(inplace = True)
    new_df = new_df.drop(['x_coordinate','y_coordinate','updated_on', 'beat', 'iucr'],axis=1)
    new_df['latitude'] = pd.to_numeric(new_df['latitude'], errors = 'coerce')
    new_df=new_df.drop(['location'],axis=1)
    return new_df

In [128]:
def crime_data_to_sql(new_df,city_table):
    
    api_key='pk.eyJ1IjoiZ2VvcmdpYWZiaSIsImEiOiJja25na20zcmwwN2l2MnFwa3BrZ2JmaHdxIn0.5aZk4GmJsmPQMAgpdia8lQ'
    new_df["zip_code"]=""

    for index, row in new_df.iterrows():
        lt=row['latitude']
        lng=row['longitude']
        crime=row['primary_type']
    #     date=pd.to_datetime(row['date']).strftime("%m/%d/%Y %H:%M:%S")
        # Run a request to endpoint and convert result to json
        target_url= f"https://api.mapbox.com/geocoding/v5/mapbox.places/{lng}%2C%20{lt}.json?access_token={api_key}"
        geo_data = requests.get(target_url).json()
        new_df.loc[index,'zip_code']=geo_data['features'][2]['text']
    #     new_df.loc[index,'date']=date


    new_df['zip_code'] = pd.to_numeric(new_df['zip_code'], errors = 'coerce')
    # new_df['zip_code']=new_df['zip_code'].astype(int)
    new_df.dropna(inplace=True)
    new_df["zip_code"]=new_df.zip_code.astype(int)

    

    user_name = 'postgres'
    pword = 'Baggins89'
    connection_string = f"postgres:{pword}@localhost:5432/crime_db"
    engine = create_engine(f'postgresql://{connection_string}')

    new_df.to_sql(name=city_table,con=engine,if_exists='replace',index=True)

    sqlite_engine=create_engine(f'sqlite:///crime_table_schema.sqlite')
    new_df.to_sql(name=city_table,con=sqlite_engine,if_exists='replace',index=True)
    
    engine.dispose()
    sqlite_engine.dispose()

In [129]:
data_frame= atlanta_df()

In [130]:
data_frame

,rpt_date,block,primary_type,location_description,latitude,longitude
id,,,,,,
200012092,1/1/2020,275 BAKER ST NW,LARCENY-FROM VEHICLE,Downtown,33.76206,-84.39568
200012063,1/1/2020,1080 W PEACHTREE ST NW,LARCENY-FROM VEHICLE,Midtown,33.78470,-84.38817
200012043,1/1/2020,1766 HOWELL MILL RD NW,BURGLARY,Underwood Hills,33.80243,-84.41529
200012040,1/1/2020,327 PETERS ST SW,LARCENY-FROM VEHICLE,Castleberry Hill,33.74586,-84.40382
200012000,1/1/2020,2400 PARKLAND DR NE,LARCENY-FROM VEHICLE,Lindbergh/Morosgo,33.82000,-84.36510
...,...,...,...,...,...,...
200140632,1/14/2020,835 MARTIN LUTHER KING JR DR,LARCENY-NON VEHICLE,Vine City,33.75504,-84.41457
200140414,1/14/2020,1039 GRANT ST SE,LARCENY-FROM VEHICLE,Grant Park,33.72613,-84.37683
200140368,1/14/2020,948 DILL AVE SW,AUTO THEFT,Capitol View,33.71823,-84.41951


Index(['AGG ASSAULT', 'AUTO THEFT', 'BURGLARY', 'HOMICIDE',
       'LARCENY-FROM VEHICLE', 'LARCENY-NON VEHICLE', 'MANSLAUGHTER',
       'ROBBERY'],
      dtype='object', name='primary_type')

In [132]:
crime_data_to_sql(atlanta_df(),'atlanta')

In [134]:
# Create engine using the `demographics.sqlite` database file
def city_crime_dict(city):
    user_name = 'postgres'
    pword = 'Baggins89'
    connection_string = f"postgres:{pword}@localhost:5432/crime_db"
    connected_engine=create_engine(f'postgresql://{connection_string}').connect()

    chicago_crime_df=pd.read_sql_table(city,connected_engine).set_index('id')


    json_result=chicago_crime_df.to_json(orient="table")
    parsed=json.loads(json_result)
    return parsed['data']

In [135]:
print(city_crime_dict('atlanta'))

In [136]:
def crime_dict():
    pword = 'Baggins89'
    connection_string = f"postgres:{pword}@localhost:5432/crime_db"
    connected_engine=create_engine(f'postgresql://{connection_string}').connect()

    chicago_crime_df=pd.read_sql_table('chicago',connected_engine).set_index('id')
    atlanta_crime_df=pd.read_sql_table('atlanta',connected_engine).set_index('id')

    json_result_chicago=chicago_crime_df.to_json(orient="table")
    json_result_atlanta=atlanta_crime_df.to_json(orient="table")
    chicagoDict=json.loads(json_result_chicago)['data']
    atlDict=json.loads(json_result_atlanta)['data']
    allCrimeDict=[]
    for d in atlDict:
        allCrimeDict.append(d)
    for c in chicagoDict:
        allCrimeDict.append(c)
    return allCrimeDict

In [137]:
print(crime_dict())

[{'id': 200012092, 'rpt_date': '1/1/2020', 'block': '275 BAKER ST NW', 'primary_type': 'LARCENY-FROM VEHICLE', 'location_description': 'Downtown', 'latitude': 33.76206, 'longitude': -84.39568, 'zip_code': 30313}, {'id': 200012063, 'rpt_date': '1/1/2020', 'block': '1080 W PEACHTREE ST NW', 'primary_type': 'LARCENY-FROM VEHICLE', 'location_description': 'Midtown', 'latitude': 33.7847, 'longitude': -84.38817, 'zip_code': 30309}, {'id': 200012043, 'rpt_date': '1/1/2020', 'block': '1766 HOWELL MILL RD NW', 'primary_type': 'BURGLARY', 'location_description': 'Underwood Hills', 'latitude': 33.80243, 'longitude': -84.41529, 'zip_code': 30318}, {'id': 200012040, 'rpt_date': '1/1/2020', 'block': '327 PETERS ST SW', 'primary_type': 'LARCENY-FROM VEHICLE', 'location_description': 'Castleberry Hill', 'latitude': 33.74586, 'longitude': -84.40382, 'zip_code': 30313}, {'id': 200012000, 'rpt_date': '1/1/2020', 'block': '2400 PARKLAND DR NE', 'primary_type': 'LARCENY-FROM VEHICLE', 'location_description